In [1]:
from argparse import Namespace

import numpy as np
import torch
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch import nn
from tqdm.notebook import tqdm, trange

from ltr.utils import seed

# Assignment 3: Learning to Rank <a class="anchor" id="top"></a>

[Back to top](#top)

In this assignment, we will implement several Learning to Rank (LTR) algorithms. We will start with offline LTR algorithms, which are trained on a dataset of precomputed features. We will then move to counterfactual LTR algorithms, which are trained on a dataset of user interactions.

Table of contents:
- [Chapter 1: Offline LRT](#o_LTR)
    - [Section 1: Feature Extraction](#feature_extraction)
    - [Section 2: Pointwise LRT](#pointwise_LTR)
    - [Section 3: Pairwise LRT](#pairwise_LTR)
    - [Section 4: Pairwise - Speed-up RankNet](#pairwise_speedup)
    - [Section 5: Listwise LRT](#listwise_LTR)
- [Chapter 2: Counterfactual LRT](#c_LTR)
    - [Section 1: Utils](#utils)
    - [Section 2: ListNet](#listnet)
    - [Section 3: Unbiased ListNet](#unbiased_listnet)

---
# Chapter 1: Offline LTR <a class="anchor" id="o_LTR"></a>

A typical setup of learning to rank involves a feature vector constructed using a query-document pair, and a set of relevance judgements. In this assignment, you are given a small dataset that consists of number of queries with their corresponding list of documents. As the first step, you need to extract features for this dataset, and then use them in order to perform learninig-to-rank with different loss functions. In particular, the information about each of the files in this dataset is as follows:

- `collection.tsv`: Each line consists of _document ID_ and the _document text_

- `queries.tsv`: Each line consists of _query ID_ and the _query text_

- `(train/dev/test)_pairs_graded.tsv`: Each line consists of _query ID_, _document ID_, _relevance score_; where relevance grade is an *ordinal* variable  with  5  grades,  for example: {`perfect`,`excellent`,`good`,`fair`,`bad`).

## Section 1: Feature Extraction <a class="anchor" id="feature_extraction"></a>

[Back to top](#top)

The following cell contains the path of raw dataset files and a file with the list of stop words. 



In [2]:
# IMPORTANT: DO NOT CHANGE ANY OF THE FILE PATHS.

COLLECTION_PATH = "./data/collection.tsv"
QUERIES_PATH = "./data/queries.tsv"
TRAIN_PATH = "./data/train_pairs_graded.tsv"
DEV_PATH = "./data/dev_pairs_graded.tsv"
TEST_PATH = "./data/test_pairs_graded.tsv"
STOP_WORDS_PATH = "./data/common_words"

First, you need to preprocess query and document texts and extract term based statistics for documents.

In [3]:
from ltr.dataset import Queries, Preprocess

prp = Preprocess(STOP_WORDS_PATH)

queries = Queries(prp)
queries.preprocess_queries(QUERIES_PATH)

In the next cell, you will use `Documents` class to extract document term-based stats, as this process might take longer, `if` statement and  `RESET` allows you to skip this step after the first time.

Complete the implementation of `process_documents` function. you can find the implemenation in `ltr.dataset`.

In [4]:
# ToDo: Implement the function 'process_documents'

from ltr.dataset import Documents

In [5]:
# IMPORTANT: DO NOT CHANGE DOC_JSON PATH
DOC_JSON = "./datasets/doc.pickle"
import os
import pickle

RESET = False
if os.path.exists(DOC_JSON) and not RESET:
    with open(DOC_JSON, "rb") as file:
        documents = pickle.load(file)
else:
    documents = Documents(prp)
    documents.process_documents(COLLECTION_PATH)
    with open(DOC_JSON, "wb") as file:
        pickle.dump(documents, file)

The class `FeatureExtraction` in `ltr.dataset` is defined for extracting features. This class includes a class `extract` method, that will extract predefined features for each query-document pair.

The other class, `GenerateFeatures` is already implemented to allow for reading lines of `(train\dev\test)_pairs_graded.tsv` files one by one, extract the features, and write them in the `(train\dev\test)_pairs_graded.tsv`. 

The `FeatureExtraction` class has the following property: `features` that is a `dict` with feature names as the keys and their values as dictionary values. The list of features that you have to implement can be found in `__feature_list__` inside `ltr.dataset` and also listed in `README.md` file.

In [6]:
# IMPORTANT: DO NOT CHANGE `args` VALUES FOR BM25 CALCULATION
N_FEATURES = 15

from ltr.dataset import FeatureExtraction, GenerateFeatures

feature_ex = FeatureExtraction({}, documents, queries)

feat_gen = GenerateFeatures(feature_ex)
args = {}
args["k1"] = 1.5
args["b"] = 0.75
args["idf_smoothing"] = 0.5

**ToDo:**

Implement method `extract` in `ltr.dataset.FeatureExtraction`. This method should return a dictionary with feature names as the keys and their values as dictionary values. This method is called inside `run` method of `GenerateFeatures` class. You can pass any required arguments through `args`. For instance, as defined in the previous cell, you can pass `k1` and `b` as BM25 arguments through `args`.

In [7]:
# feat_gen.run(TRAIN_PATH, TRAIN_PATH + "g", **args)
# feat_gen.run(DEV_PATH, DEV_PATH + "g", **args)
# feat_gen.run(TEST_PATH, TEST_PATH + "g", **args)

The `DataSet` class will read the generated feature files for train/valid/test splits and you can use them for training LTR models with different ranking loss functions.

In [8]:
# IMPORTANT: DO NOT CHANGE THE `fold_paths`

from ltr.dataset import DataSet

fold_paths = ["./data/"]
num_relevance_labels = 5
num_nonzero_feat = N_FEATURES
num_unique_feat = N_FEATURES
data = DataSet(
    "ir1-2023", fold_paths, num_relevance_labels, num_unique_feat, num_nonzero_feat
)

data = data.get_data_folds()[0]
data.read_data()

In [9]:
print(f"Number of features: {data.num_features}")
# print some statistics
for split in ["train", "validation", "test"]:
    print(f"Split: {split}")
    split = getattr(data, split)
    print(f"\tNumber of queries {split.num_queries()}")
    print(f"\tNumber of docs {split.num_docs()}")

Number of features: 15
Split: train
	Number of queries 961
	Number of docs 89506
Split: validation
	Number of queries 195
	Number of docs 19500
Split: test
	Number of queries 483
	Number of docs 48127


### Section 1.2 Utility classes/methods

You can use `LTRData` class in this assignment that is defined in `ltr.dataset`. 

In [10]:
from ltr.dataset import LTRData

## example
train_dl = DataLoader(LTRData(data, "train"), batch_size=32, shuffle=True)

# this is how you would use it to quickly iterate over the train/val/test sets
# - (of course, without the break statement!)
for x, y in train_dl:
    print(x.size(), y.size())
    break

torch.Size([32, 15]) torch.Size([32])


You can use `evaluate_model` from `ltr.eval` to evaluate a model, on a given split.

In [11]:
from ltr.eval import evaluate_model

## example
# function that scores a given feature vector e.g a network
net = nn.Linear(N_FEATURES, 1)


# the evaluate method accepts a function. more specifically, a callable (such as pytorch modules)
def notwork(x):
    return net(x)


# evaluate the function
_ = evaluate_model(data, notwork, split="validation", print_results=True)

Eval (validation):   0%|          | 0/77 [00:00<?, ?it/s]

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


"metric": "mean" ("standard deviation")
dcg: 6.1153 (8.63432)
dcg@03: 0.5122 (2.16958)
dcg@05: 0.6263 (2.44159)
dcg@10: 0.8664 (2.74802)
dcg@20: 1.3148 (3.48596)
ndcg: 0.2292 (0.11308)
ndcg@03: 0.0234 (0.10689)
ndcg@05: 0.0250 (0.10695)
ndcg@10: 0.0326 (0.11167)
ndcg@20: 0.0479 (0.12391)
precision@01: 0.0165 (0.12733)
precision@03: 0.0165 (0.07227)
precision@05: 0.0143 (0.05943)
precision@10: 0.0132 (0.04248)
precision@20: 0.0124 (0.03228)
recall@01: 0.0086 (0.08257)
recall@03: 0.0222 (0.13329)
recall@05: 0.0269 (0.13940)
recall@10: 0.0482 (0.18088)
recall@20: 0.0874 (0.24332)
relevant rank: 68.0023 (30.36636)
relevant rank per query: 164.7747 (189.37578)


The next cell is used to generate reproducible results:

In [12]:
# use to get reproducible results
from ltr.utils import seed

## Section 2: Pointwise LTR <a class="anchor" id="pointwise_LTR"></a>

[Back to top](#top)

Let $x \in \mathbb{R}^d$ be an input feature vector, containing features for a query-document pair. Let $f: \mathbb{R}^d \rightarrow \mathbb{R} $ be a function that maps this feature vector to a number $f(x)$ - either a relevance score (regression) or label (classification). The data $\{x \}$ are treated as feature vectors and the relevance judgements are treated as the target which we want to predict. 

In this section, you will implement a simple Pointwise model using either a regression loss, and use the train set to train this model to predict the relevance score. 

### Section 2.1: Neural Model


\# ToDO:
Implement the following methods for the Class `LTRModel` in `ltr.model`:
  - `__init__`
  - `forward`

We will use this neural network to learn a LTR model with different loss functions(Pointwise, Pairwise, Listwise), using the relevance grades as the label. This network should have the following attribute:
  - `layers`: $N_{FEATURES} (input) \rightarrow 10 \rightarrow 1$ where each layer is a linear layer (`nn.Linear`) with a ReLu activation function (`nn.ReLU`) in between the layers. Use the default weight initialization scheme. (Hint: use `nn.Sequential` for a one-line forward function!)

In [13]:
from ltr.model import LTRModel

# check the network configuration - layer dimensions and configurations
point_nn_reg = LTRModel(data.num_features)
print(point_nn_reg)

LTRModel(
  (layers): Sequential(
    (input): Linear(in_features=15, out_features=10, bias=True)
    (relu): ReLU()
    (output): Linear(in_features=10, out_features=1, bias=True)
  )
)


\#ToDo: Implement regression loss in `pointwise_loss` function. You can find the definition in `ltr.ltr`.

In [14]:
from ltr.loss import pointwise_loss

\# ToDO:
Implement `train_pointwise` function in `ltr.ltr` as a wrapper for training a pointwise LTR, that takes the model as input and trains the model.

In [15]:
from ltr.train import train_pointwise

In [16]:
# Change this to test your code!
pointwise_test_params = Namespace(epochs=1, lr=1e-3, batch_size=256, metrics={"ndcg"})

In [17]:
# train a regression model for testing (we are only training for 1 epoch)
# TODO: uncomment later!
met_reg = train_pointwise(point_nn_reg, pointwise_test_params, data)

Training Epoch 1:   9%|▉         | 31/350 [00:00<00:02, 148.61it/s]

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Training Epoch 1:  27%|██▋       | 96/350 [00:00<00:00, 255.15it/s]

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Training Epoch 1:  37%|███▋      | 128/350 [00:00<00:00, 275.25it/s]

...

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Training Epoch 1:  91%|█████████ | 317/350 [00:01<00:00, 268.09it/s]

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Training Epoch 1: 100%|██████████| 350/350 [00:01<00:00, 220.52it/s]


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


The function `create_results` is defined in `ltr.utils` to create the results. It could be used  to generate your results:

In [18]:
# IMPORTANT DO NOT CHANGE seed, params, or any of the PATH variables
from ltr.utils import create_results

seed(42)

params_regr = Namespace(
    epochs=11, lr=1e-3, batch_size=1, metrics={"ndcg", "precision@05", "recall@05"}
)

# TODO: uncomment later!
pointwise_regression_model = LTRModel(data.num_features)
pw_regr = create_results(
     data,
     pointwise_regression_model,
     train_pointwise,
     pointwise_regression_model,
     "./outputs/pointwise_res.json",
     params_regr,
)

torch.save(pointwise_regression_model.state_dict(), "./outputs/pointwise_model")

Training Model


Training Epoch 11: 100%|██████████| 89506/89506 [01:12<00:00, 1226.72it/s]


	"precision@05": (0.26681514476614704, 0.19840985710511272)
	"recall@05": (0.756355342402768, 0.36479750387633053)
	"ndcg": (0.802665563913099, 0.24760109553815426)


## Section 3: Pairwise LTR <a class="anchor" id="pairwise_LTR"></a>

[Back to top](#top)

In this section,  you will learn and implement RankNet, a  pairwise learning to rank algorithm.

For a given query, consider two documents $D_i$ and $D_j$ with two different ground truth relevance  labels,  with  feature  vectors $x_i$ and $x_j$ respectively.   The  RankNet  model,  just  like  the pointwise model, uses $f$ to predict scores i.e $s_i=f(x_i)$ and $s_j=f(x_j)$, but uses a different loss during  training. $D_i \triangleright D_j$ denotes  the  event  that $D_i$ should  be  ranked  higher  than $D_j$.   The  two outputs $s_i$ and $s_j$ are mapped to a learned probability that $D_i \triangleright D_j$: 


$$        P_{ij} = \frac{1}{1 + e^{-\sigma(s_i - s_j)}} $$
  
where $\sigma$ is a parameter that determines the shape of the sigmoid. The loss of the RankNet model is the cross entropy cost function:

$$        C = - \bar{P}_{ij} \log P_{ij} - (1-\bar{P}_{ij}) \log (1 - P_{ij}) $$

As the name suggests, in the pairwise approach to LTR, we optimize a loss $l$ over pairs of documents. Let $S_{ij} \in \{0, \pm1 \}$ be equal to $1$ if the relevance of document $i$ is greater than document $j$; $-1$ if document $j$ is more relevant than document $i$; and 0 if they have the same relevance. This gives us $\bar{P}_{ij} = \frac{1}{2} (1 + S_{ij})$ so that $\bar{P}_{ij} = 1$ if $D_i \triangleright D_j$; $\bar{P}_{ij} = 0$ if $D_j \triangleright D_i$; and finally $\bar{P}_{ij} = \frac{1}{2}$ if the relevance is identical. This gives us:

$$        C = \frac{1}{2}(1- S_{ij})\sigma(s_i - s_j) + \log(1+ e^{-\sigma(s_i - s_j)}) $$

Now, consider a single query for which $n$ documents have been returned. Let the output scores of the ranker be $s_j$ ; $j=\{1, \dots, n \}$, the model parameters be $w_k \in \mathbb{R}^W$, and let the set of pairs of document indices used for training be $\mathcal{P}$. Then, the total cost is $C_T = \sum_{i,j \in \mathcal{P}} C(s_i; s_j)$. 



- Implement RankNet. You should construct training samples by creating all possible pairs of documents for a given query and optimizing the loss above.
- Use the same model architecture as in pointwise LTR.

For the pairwise loss, we need to have a structured **dataloader** which detects the documents associated with a specific query. The class `QueryGroupedLTRData` is defined in `ltr.dataset` for this end.

In [19]:
from ltr.dataset import QueryGroupedLTRData
from ltr.utils import create_results

In [20]:
## example
train_data = QueryGroupedLTRData(data, "train")
# this is how you would use it to quickly iterate over the train/val/test sets

q_i = 300
features_i, labels_i = train_data[q_i]
print(f"Query {q_i} has {len(features_i)} query-document pairs")
print(f"Shape of features for Query {q_i}: {features_i.size()}")
print(f"Shape of labels for Query {q_i}: {labels_i.size()}")

Query 300 has 100 query-document pairs
Shape of features for Query 300: torch.Size([100, 15])
Shape of labels for Query 300: torch.Size([100])


#ToDO:

First, implement the pairwaise loss in `pairwise_loss` function in `ltr.loss` as described above.
Then, implement `train_pairwise` method in `ltr.train`.

In [21]:
from ltr.loss import pairwise_loss
from ltr.train import train_pairwise

Normal training with pairwise loss requires a lot of time, you can investigate normal training of LTRModel with pairwise loss by uncommenting the following cell.

In [22]:
example_batch_size = 2 # NOTE: they provided 1 as an example
seed(42)


params = Namespace(
    epochs=1,
    lr=1e-4,
    batch_size=example_batch_size,
    metrics={"ndcg", "precision@05", "recall@05"},
)


pairwise_net = LTRModel(N_FEATURES)

# TODO: uncomment later!
create_results(
     data, pairwise_net,
     train_pairwise,
     pairwise_net,
     "./outputs/pairwise_normal_res.json",
     params

 )

torch.save(pairwise_net.state_dict(), "./outputs/pairwise_normal_model")

Training Model


Training Epoch 1: 100%|██████████| 44753/44753 [00:56<00:00, 791.72it/s]


	"precision@05": (0.25478841870824054, 0.19556442726397166)
	"recall@05": (0.7230200087988637, 0.3884456344005742)
	"ndcg": (0.7833490442592383, 0.2670017311523801)


## Section 4: Pairwise: Speed-up RankNet <a class="anchor" id="pairwise_speedup"></a>

[Back to top](#top)

To speed up training of the previous model, we can consider a sped up version of the model, where instead of `.backward` on the loss, we use `torch.backward(lambda_i)`. 

The derivative of the total cost $C_T$ with respect to the model parameters $w_k$ is:

$$        \frac{\partial C_T}{\partial w_k} = \sum_{(i,j) \in \mathcal{P}} \frac{\partial C(s_i, s_j)}{\partial s_i} \frac{\partial s_i}{\partial w_k} + \frac{\partial C(s_i, s_j)}{\partial s_j} \frac{\partial s_j}{\partial w_k} $$

We can rewrite this sum by considering the set of indices $j$ , for which $\{i,j\}$ is a valid pair, denoted by $\mathcal{P}_i$, and the set of document indices $\mathcal{D}$:

$$
\frac{\partial C_T}{\partial w_k} = \sum_{i \in \mathcal{D}}
\frac{\partial s_i}{\partial w_k} \sum_{j \in \mathcal{P}_i} 
\frac{\partial C(s_i, s_j)}{\partial s_i} 
$$

This sped of version of the algorithm first computes scores $s_i$ for all the documents. Then for each $j= 1, \dots, n$, compute:

$$
\lambda_{ij} = \frac{\partial C(s_i, s_j)}{\partial s_i} = \sigma \bigg( \frac{1}{2}(1 - S_{ij}) -  \frac{1}{1 + e^{\sigma(s_i -s_j))}} \bigg) \\
\lambda_i = \sum_{j \in \mathcal{P}_i} \frac{\partial C(s_i, s_j)}{\partial s_i} = \sum_{j \in \mathcal{P}_i} \lambda_{ij}
$$

That gives us:

$$
\frac{\partial C_T}{\partial w_k} = \sum_{i \in \mathcal{D}}
\frac{\partial s_i}{\partial w_k} \lambda_i
$$

This can be directly optimized in pytorch using: `torch.autograd.backward(scores, lambda_i)` 
 


\# ToDO:

Implement the `compute_lambda_i` for the sped-up version of pairwise loss as described above. You can find the function definiton in `ltr.ltr`.

In [23]:
from ltr.loss import compute_lambda_i

In [24]:
from ltr.loss import mean_lambda

\# ToDO:

Implement `train_pairwise_spedup` function for more efficient training with pairwise loss. You can find the definition in `ltr.train`. 

In [25]:
from ltr.train import train_pairwise_spedup

In [26]:
# IMPORTANT DO NOT CHANGE seed, params, or any of the PATH variables
from ltr.utils import create_results
seed(42)

params = Namespace(
    epochs=11, lr=1e-3, batch_size=1, metrics={"ndcg", "precision@05", "recall@05"}
)

sped_up_pairwise_model = LTRModel(N_FEATURES)

# TODO: uncomment later!
create_results(
     data,
     sped_up_pairwise_model,
     train_pairwise_spedup,
     sped_up_pairwise_model,
     "./outputs/pairwise_spedup_res.json",
     params,
)

torch.save(sped_up_pairwise_model.state_dict(), "./outputs/pairwise_spedup_model")

Training Model


	"precision@05": (0.2717149220489978, 0.19871961820437448)
	"recall@05": (0.7637550510831004, 0.3556460610701)
	"ndcg": (0.7978669386934512, 0.248687517014588)


##  Section 5: Listwise LTR <a class="anchor" id="listwise_LTR"></a>

[Back to top](#top)

In this section, you will implement LambdaRank, a listwise approach to LTR. Consider the computation of $\lambda$ for sped-up RankNet (that you've already implemented). $\lambda$ here amounts to the 'force' on a document given its neighbours in the ranked list. The design of $\lambda$ in LambdaRank is similar to RankNet, but is scaled by DCG gain from swapping the two documents in question. Let's suppose that the corresponding ranks of doucment $D_i$ and $D_j$ are $r_i$ and $r_j$ respectively. Given a ranking measure $IRM$, such as $NDCG$ or $ERR$, the lambda function in LambdaRank is defined as:


$$        \frac{\partial C}{\partial s_i} = \sum_{j \in D} \lambda_{ij} \cdot |\bigtriangleup IRM (i,j)| $$

Where $|\bigtriangleup IRM(i,j)|$ is the absolute difference in $IRM$ after swapping the rank positions $r_i$ and $r_j$ while leaving everything else unchanged ($| \cdot |$ denotes the absolute value). Note that we do not backpropogate $|\bigtriangleup IRM|$, it is treated as a constant that scales the gradients. In this assignment we will use $|\bigtriangleup NDCG|$

\# ToDO:

Implement the listwise loss in `listwise_loss` function defined in `ltr.ltr`



In [14]:
from ltr.loss import listwise_loss

The function `mean_lambda_list` is defined in `ltr.ltr`.

\# ToDo:
Implement `train_listwise` function to train with list_wise loss. you can find the definition in `ltr.train`.

In [15]:
from ltr.loss import mean_lambda_list
from ltr.train import train_listwise

In [16]:
# IMPORTANT DO NOT CHANGE seed, params, or any of the PATH variables
from ltr.utils import create_results
seed(42)

params = Namespace(
    epochs=11, lr=1e-4, batch_size=1, metrics={"ndcg", "precision@05", "recall@05"}
)

listwise_model = LTRModel(N_FEATURES)

# TODO: uncomment later!
create_results(
    data,
    listwise_model,
    train_listwise,
    listwise_model,
    "./outputs/listwise_res.json",
    params,
)

torch.save(listwise_model.state_dict(), "./outputs/listwise_model")

Training Model


Training Epoch 11: 100%|██████████| 961/961 [00:01<00:00, 633.27it/s]


	"recall@05": (0.7730315948808023, 0.3508352072096778)
	"ndcg": (0.8127174942598598, 0.2451201100659698)
	"precision@05": (0.2739420935412027, 0.1972253100579782)


---
# Chapter 2: Counterfactual LTR <a class="anchor" id="c_LTR"></a>

In this chapter, you will implement counterfactual learning to rank algorithms. In contrast to offline LTR algorithms, counterfactual LTR algorithms are trained on a dataset of user interactions. First, we will load the dataset:

In [17]:
from ltr.dataset import load_data

data = load_data()

We assume that there is a logging policy that shows the results for each query to the users and logs the user clicks.
For that, we provide a logging policy simulator `LoggingPolicy`.
Our logging policy only shows top 20 documents to the users.
You can use this simulator to:
- Get the position of the documents for a query in the SERP: `query_positions`.
- Gather the (simulated) clicks of users for a query: `gather_clicks`.

In [18]:
from ltr.logging_policy import LoggingPolicy

logging_policy = LoggingPolicy(policy_path="data/")

# Gather the clicks on the SERP for query 20
for i in range(10):
    clicked_docs = np.where(logging_policy.gather_clicks(20))[0]
    clicked_positions = logging_policy.query_positions(20)[clicked_docs]
    print(
        f"clicks for session {i+1} on documents",
        clicked_docs,
        "on positions",
        clicked_positions,
    )

clicks for session 1 on documents [32 99] on positions [3 0]
clicks for session 2 on documents [ 0 99] on positions [17  0]
clicks for session 3 on documents [32 33 99] on positions [3 1 0]
clicks for session 4 on documents [33 99] on positions [1 0]
clicks for session 5 on documents [33 99] on positions [1 0]
clicks for session 6 on documents [32 99] on positions [3 0]
clicks for session 7 on documents [33 99] on positions [1 0]
clicks for session 8 on documents [33 99] on positions [1 0]
clicks for session 9 on documents [33 69 83 87 98 99] on positions [ 1  7 11 16  4  0]
clicks for session 10 on documents [32 33 99] on positions [3 1 0]


## Section 1: Utils <a class="anchor" id="utils"></a>

[Back to top](#top)

### Click data loader
First, we need to have a data loader that feeds the model with features and click data.
In this data loader, you have to select `topk=20` items for each query, and return three tensors:
- Feature vectors of the selected documents,
- One instance of the clicks over the selected documents, using the `gather_clicks(qid)` function, and
- The positions of the selected documents in the SERP.

**IMPORTANT** Here you *should not* use the `labels` for training. It is assumed that we cannot observe the real labels and want to use the `clicks` to train our LTR model 
instead.

ToDo: Implement the `__getitem__` method in `ClickLTRData` class in `ltr.dataset` to return the feature vectors, clicks, and positions of the selected documents for each query.

In [19]:
from ltr.dataset import ClickLTRData

train_dl = DataLoader(ClickLTRData(data, logging_policy), batch_size=1, shuffle=True)

for features, clicks, positions in train_dl:
    print(features.shape, clicks.shape, positions.shape)

    assert positions.dtype == torch.long
    print(features.shape, clicks.shape, positions.shape)
    print("clicks:", clicks)
    print("positions:", positions)
    break

torch.Size([1, 20, 15]) torch.Size([1, 20]) torch.Size([1, 20])
torch.Size([1, 20, 15]) torch.Size([1, 20]) torch.Size([1, 20])
clicks: tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
         0., 1.]])
positions: tensor([[15,  0,  1, 17, 14, 10, 13,  7,  3,  2, 18,  6,  8, 19,  9, 16,  4, 11,
         12,  5]])


### LTR model
Further, let's copy the idea of the `LTRModel` from previous chapter to implement `CLTRModel` in `ltr.model` for counterfactual learning to rank and take the width of the middle layer as an argument.

ToDo: Implement the `__init__` and `forward` methods in `CLTRModel` class in `ltr.model`.

In [20]:
from ltr.model import CLTRModel

net = CLTRModel(data.num_features, width=20)
print(net)

CLTRModel(
  (layers): Sequential(
    (input): Linear(in_features=15, out_features=20, bias=True)
    (relu): ReLU()
    (output): Linear(in_features=20, out_features=1, bias=True)
  )
)


---

## Section 2: ListNet <a class="anchor" id="listnet"></a>

In the previous chapter, you have implemented different loss functions for LTR.
Here we use another well known listwise loss funtion, called `ListNet`, and will use it for our unbiased LTR model.
The idea behind ListNet is very simple:
To solve the discontinuity issue of NDCG, in **ListNet**, the loss function is based on probability distribution on permutations.

Define a family of distributions on permutation of scores $z$, $P_z(\pi)$, s.t. $\sum_{\pi\in\Omega} P_z(\pi)=1$, where $\Omega$ is the set of all $n!$ permutations.
Ideally, we want the scores of our LTR model lead to the same permutation distribution as the labels $y$, i.e.,

$$
\min KL(P_y,P_z)=-\sum_{\pi\in\Omega} P_y(\pi) \log P_z(\pi)
$$

Plackett-Luce distribution gives a general formula for calculating the permutation distribution:

$$
P_z(\pi) = \prod_{j=1}^{n} \frac{\exp(z_{\pi(j)})}{\sum_{k=j}^{n} \exp(z_{\pi(k)})}
$$
In ListNet, instead of calculating $n!$ permutation probabilities, the top one probability of each document is calculated:

$$
P_z(j) = \sum_{\pi(1)=j, \pi\in\Omega} P_z(\pi) = \frac{\exp(z_{j})}{\sum_{k=1}^{n} \exp(z_{k})},
$$
which is the softmax function.

Then, the loss is defined as follows:

$$
\mathcal{L}_{\text{ListNet}}=-\sum_{j=1}^{n} P_y(j) \log P_z(j),
$$
where the softmax function is used to calculate $P_y(j)$ and $P_z(j)$ from the labels and predictions, respectively.

### ListNet loss function

ToDo: Implement the ListNet loss function.

In [21]:
from ltr.loss import listNet_loss

biased_net = CLTRModel(data.num_features, width=20)

for features, clicks, positions in train_dl:
    print(features.shape, clicks.shape, positions.shape)
    output = biased_net(features)
    print(output.shape, clicks.shape)
    loss = listNet_loss(output, clicks)
    print(loss)
    break

torch.Size([1, 20, 15]) torch.Size([1, 20]) torch.Size([1, 20])
torch.Size([1, 20, 1]) torch.Size([1, 20])
tensor(3.5748, grad_fn=<NegBackward0>)


### Biased ListNet training
Now use `listNet_loss` to train an LTR model. Since we use `clicks` instead of `relevance`, and do not correct for the bias, this would be a biased model.

In [22]:
from ltr.train import train_biased_listNet

policy_path = "./data/"

params = Namespace(
    epochs=1,
    lr=1e-4,
    batch_size=1,
    metrics={"ndcg@10", "precision@10", "recall@10"},
    policy_path=policy_path,
)

biased_net = CLTRModel(15, width=20)
train_biased_listNet(biased_net, params, data)

100%|██████████| 1/1 [00:01<00:00,  1.58s/it]


{'metrics_val': [{'dcg': (6.407137904743902, 8.605644802098313),
   'dcg@03': (0.9069527446886535, 3.182057496581204),
   'dcg@05': (1.0264029853885868, 3.2935147804952427),
   'dcg@10': (1.267604343542305, 3.454953164538368),
   'dcg@20': (1.7395199521799476, 4.002964761871101),
   'ndcg': (0.24777274568051175, 0.1532241336670187),
   'ndcg@03': (0.04698333256323492, 0.17521309349486086),
   'ndcg@05': (0.05085293221892146, 0.17554552466385964),
   'ndcg@10': (0.056426204392010196, 0.17561305050093493),
   'ndcg@20': (0.07173416162569271, 0.1788072263212249),
   'precision@01': (0.04395604395604396, 0.20499734182612783),
   'precision@03': (0.02747252747252747, 0.09810081861422768),
   'precision@05': (0.02197802197802198, 0.06922204724405988),
   'precision@10': (0.016483516483516484, 0.043887308896078815),
   'precision@20': (0.015384615384615385, 0.03330866937632456),
   'recall@01': (0.026613130459284307, 0.15139386298604413),
   'recall@03': (0.05041076139977238, 0.20862651058926

### Saving the results
Since we randomly simulate clicks and use them to train our model, for the evaluation we train and save 10 different models and inspect the average and std over them.

**IMPORTANT** Run the following cell to store your models and results. After it finishes, make sure to push the results to the git repo.

_Estimated time on Codespaces_: 5m

In [23]:
from ltr.utils import create_results
from ltr.train import train_biased_listNet

# TODO: uncomment later!
seed(42)
params = Namespace(
     epochs=20,
     lr=1e-4,
     batch_size=1,
     metrics={"ndcg@10", "precision@10", "recall@10"},
     policy_path="data/",
)


for i in range(10):
     print("Training Model", i)
     biased_net = CLTRModel(15, width=20)
     create_results(
         data,
         biased_net,
         train_biased_listNet,
         biased_net,
         f"./outputs/biased_listNet_{i}.json",
         params,
     )

torch.save(biased_net.state_dict(), f"./outputs/biased_listNet_{i}")

Training Model 0
Training Model


100%|██████████| 20/20 [00:29<00:00,  1.46s/it]


	"recall@10": (0.8145967763549318, 0.3247335674062208)
	"precision@10": (0.15478841870824056, 0.13173677016764646)
	"ndcg@10": (0.6936378981416389, 0.3221915447453277)
Training Model 1
Training Model


100%|██████████| 20/20 [00:27<00:00,  1.39s/it]


	"recall@10": (0.6894102934781567, 0.4008414933356224)
	"precision@10": (0.13184855233853007, 0.12137451200243217)
	"ndcg@10": (0.5906489562679066, 0.36799529736018766)
Training Model 2
Training Model


...

	"recall@10": (0.750915666571372, 0.36762777570287286)
	"precision@10": (0.14543429844098, 0.13375791974807275)
	"ndcg@10": (0.6630109018397662, 0.34844996465016975)
Training Model 8
Training Model


100%|██████████| 20/20 [00:27<00:00,  1.40s/it]


	"recall@10": (0.795845466744851, 0.3385130881668076)
	"precision@10": (0.1492204899777283, 0.12752769183966067)
	"ndcg@10": (0.6982858754057727, 0.3349026726210293)
Training Model 9
Training Model


100%|██████████| 20/20 [00:27<00:00,  1.39s/it]


	"recall@10": (0.784302559312385, 0.34857487881439847)
	"precision@10": (0.15033407572383073, 0.1343337091314051)
	"ndcg@10": (0.6198156704312677, 0.3324738419511254)


---

## Section 3: Unbiased ListNet <a class="anchor" id="unbiased_listnet"></a>

### Unbiased ListNet loss function

Now, we use IPS to have an unbiased ListNet:

In [24]:
from ltr.loss import unbiased_listNet_loss

unbiased_net = CLTRModel(data.num_features, width=20)
propensity = logging_policy.propensity

for features, clicks, positions in train_dl:
    print(features.shape, clicks.shape, positions.shape)
    output = biased_net(features)
    print(output.shape, clicks.shape)
    loss = unbiased_listNet_loss(output, clicks, propensity[positions.data.numpy()])
    print(loss)
    break

torch.Size([1, 20, 15]) torch.Size([1, 20]) torch.Size([1, 20])
torch.Size([1, 20, 1]) torch.Size([1, 20])
tensor(3.0563, grad_fn=<NegBackward0>)


### Unbiased ListNet training
Now use `unbiased_listNet_loss` to train an LTR model.

In [25]:
from ltr.train import train_unbiased_listNet

params = Namespace(
    epochs=1,
    lr=1e-4,
    batch_size=1,
    propensity=logging_policy.propensity,
    metrics={"ndcg@10", "precision@10", "recall@10"},
    policy_path="data/"
)

biased_net = CLTRModel(15, width=20)
train_unbiased_listNet(biased_net, params, data)

100%|██████████| 1/1 [00:01<00:00,  1.41s/it]


{'metrics_val': [{'dcg': (12.365226444428973, 11.469237322675477),
   'dcg@03': (6.524804092728993, 7.9625650267799655),
   'dcg@05': (7.602576064455456, 8.21427461796692),
   'dcg@10': (9.016507687238738, 8.575564757337617),
   'dcg@20': (9.900950514876508, 9.23169458800952),
   'ndcg': (0.546926673490645, 0.2801112133432139),
   'ndcg@03': (0.3581107910258369, 0.3958922999874043),
   'ndcg@05': (0.39611793358496206, 0.37530311744999195),
   'ndcg@10': (0.44633638490120814, 0.3458734247373074),
   'ndcg@20': (0.47264441131229173, 0.3305964147850289),
   'precision@01': (0.3241758241758242, 0.46806608421861684),
   'precision@03': (0.20695970695970695, 0.24564666264729795),
   'precision@05': (0.16263736263736261, 0.1674950231495178),
   'precision@10': (0.11593406593406594, 0.10700187963818925),
   'precision@20': (0.07115384615384615, 0.07490131574673248),
   'recall@01': (0.21039377289377287, 0.36005309354148507),
   'recall@03': (0.3671310832025118, 0.4322682303595382),
   'recall@

### Saving the results
Similar to the biased model, here we train 10 different unbiased models and save them to inspect the average and std over them.

**IMPORTANT** Run the following cell to store your models and results. After it finishes, make sure to push the results to the git repo.

_Estimated time on Codespaces_: 5m

In [26]:
from ltr.utils import create_results
from ltr.train import train_unbiased_listNet

seed(42)
params = Namespace(
     epochs=20,
     lr=1e-4,
          batch_size=1,
     propensity=logging_policy.propensity,
     metrics={"ndcg@10", "precision@10", "recall@10"},
)

for i in range(10):
     print("Training Model", i)
     unbiased_net = CLTRModel(15, width=20)
     create_results(
         data,
         unbiased_net,
         train_unbiased_listNet,
         unbiased_net,
         f"./outputs/unbiased_listNet_{i}.json",
         params,
)

torch.save(unbiased_net.state_dict(), f"./outputs/unbiased_listNet_{i}")

Training Model 0
Training Model


100%|██████████| 20/20 [00:32<00:00,  1.60s/it]


	"recall@10": (0.826666324272115, 0.31480706731131686)
	"precision@10": (0.15812917594654793, 0.13623009249792625)
	"ndcg@10": (0.7433885628120946, 0.31505730098157403)
Training Model 1
Training Model


100%|██████████| 20/20 [00:33<00:00,  1.67s/it]


	"recall@10": (0.8173747664478702, 0.32402309030962334)
	"precision@10": (0.15567928730512254, 0.13388172336658907)
	"ndcg@10": (0.7345897358192324, 0.31860734429907334)
Training Model 2
Training Model


...

	"recall@10": (0.8183937527020194, 0.32605756375217326)
	"precision@10": (0.15902004454342986, 0.13956637283645396)
	"ndcg@10": (0.7271101649090158, 0.3205701034325043)
Training Model 8
Training Model


100%|██████████| 20/20 [00:29<00:00,  1.50s/it]


	"recall@10": (0.825226466838546, 0.3152024209132493)
	"precision@10": (0.1541202672605791, 0.12588110402650113)
	"ndcg@10": (0.6870241439219921, 0.31728568152323516)
Training Model 9
Training Model


100%|██████████| 20/20 [00:30<00:00,  1.53s/it]


	"recall@10": (0.836137437827468, 0.3042411798264741)
	"precision@10": (0.15946547884187087, 0.13046321205289343)
	"ndcg@10": (0.7200807387153808, 0.31068704599901303)


In [27]:
# see: https://edstem.org/eu/courses/1916/discussion/170770

import json
import numpy as np

 
def aggregate_results(model_name):
    aggregated_metrics = {}
    for i in range(10):
        with open(f"./outputs/{model_name}_{i}.json", "r") as reader:
            result = json.load(reader)
            for metric, (v, std) in result["test_metrics"].items():
                aggregated_metrics.setdefault(metric, []).append(v)
    return {metric: np.mean(vals) for metric, vals in aggregated_metrics.items()}


biased = aggregate_results("biased_listNet")
unbiased = aggregate_results("unbiased_listNet")
# save the aggregated output files
for model_avg_results, model_name in zip(
    [biased, unbiased], ["biased_listNet", "unbiased_listNet"]
):
    json.dump(model_avg_results, open(f"outputs/{model_name}_avg.json", "wt"))

# display a handful of metrics
print_metrics = ["ndcg", "ndcg@20", "precision@05", "recall@20"]
print_biased = {metric: v for metric, v in biased.items() if metric in print_metrics}
print_unbiased = {
    metric: v for metric, v in unbiased.items() if metric in print_metrics
}

import pandas as pd

pd.set_option("display.precision", 3)
df = pd.DataFrame([print_biased, print_unbiased], index=["biased", "unbiased"])
print(df)

           ndcg  ndcg@20  precision@05  recall@20
biased    0.733    0.690         0.240      0.859
unbiased  0.786    0.752         0.259      0.899
